In [1]:
!pip install AugmentedSocialScientist pandas numpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 50.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 35.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 62.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [3]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
901,820186,NaN,Mais s’il arrive que le visage que le miroir n...
360,670170,Agent responsable,"Article 1611 , alinéa 3\n“ Lorsque l’identific..."
432,1210724,Individu,Il n’en est rien. Les soins palliatifs existen...
159,130019,Agent responsable,Citons cependant les Comités de\nl'Assistance ...
1331,520008,NaN,Le développement de recherches nécessitant des...


In [4]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
298,360049,Matériel biologique,Ce vecteur viral peut être introduit\ndirectem...
672,680029,Individu,"par mois. Une telle somme, qui équivaut à pein..."
618,380022,Genre/espèce humaine,Ces Comités auraient notamment pour fonction :...
492,740121,Individu,3 - Seraitil plus éthique de confier la gestio...
180,500004,Matériel biologique,"2. La stérilisation constituet-elle une ""viole..."


In [5]:
annotations["Personne"].value_counts()

Personne
Personne à protéger       143
Matériel biologique       127
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Humain comme empathie      10
Poubelle                    8
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.90)

In [7]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [8]:
batch_size = 32
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values, batch_size=batch_size)

  0%|          | 0/627 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/627 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [9]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values, batch_size=batch_size)

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Individu': 2, 'Matériel biologique': 3, 'Personne relationnelle': 4, 'Personne à protéger': 5, 'Poubelle': 6}


In [10]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=1,
    lr=5e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 1 ========
Training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 305.56 MiB is free. Process 2300550 has 14.28 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 80.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import numpy as np
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1).dropna()
pred_loader = bert.encode(pred_data.sentence_text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}
